In [1]:
%gui asyncio

from threading import Thread
from time import sleep
from ipywidgets import GridspecLayout, Layout, Button
from IPython.display import display

GESTOPPT = True
TERMINATE = False
GRID_GROSSE = 20
MAIN_LOOP = None
ZELLEN_ZUSTAND = {}
SPIELFELD = None

def zustand_festlegen(button, zustand):
    if zustand not in ["an", "aus"]:
        raise RuntimeError("Falscher Zustand gegeben: " + zustand + " an Zelle " + str(x) + ", " + str(y))
    
    x = button.location_x
    y = button.location_y
    ZELLEN_ZUSTAND[x, y] = zustand
    if zustand == "an":
        button.style = {"button_color": "gray"}
    else:
        button.style = {"button_color": "white"}
  
def spielfeld_erstellen():
    global GESTOPPT
    zelle_grosse = 30
    spielfeld = GridspecLayout(
        GRID_GROSSE,
        GRID_GROSSE,
        width=f"{zelle_grosse * GRID_GROSSE}px",
        height=f"{zelle_grosse * GRID_GROSSE}px",
        style={"background_color": "black"}
    )

    def zustand_wechseln(button):
        if not GESTOPPT:
            return
        x = button.location_x
        y = button.location_y
        if ZELLEN_ZUSTAND[x, y] == "aus":
            zustand_festlegen(button, "an")
        else:
            zustand_festlegen(button, "aus")

    for x in range(GRID_GROSSE):
        for y in range(GRID_GROSSE):
            button = Button(
                layout=Layout(height="auto", width="auto", margin="0", border="1px solid gray"),
            )
            button.location_x = x
            button.location_y = y
            zustand_festlegen(button, "aus")
            button.on_click(zustand_wechseln)
            spielfeld[x, y] = button

    return spielfeld

def conways_spiel():
    global GESTOPPT, MAIN_LOOP, ZELLEN_ZUSTAND, TERMINATE, SPIELFELD
    GESTOPPT = True
    ZELLEN_ZUSTAND = {}
    SPIELFELD = spielfeld_erstellen()

    play_button = Button(description="Play", layout=Layout(width="80px"))
    pause_button = Button(description="Pause", disabled=True, layout=Layout(width="80px"))
    leeren_button = Button(description="Leeren", disabled=False, layout=Layout(width="80px"))

    def play(_):
        global GESTOPPT
        GESTOPPT = False
        play_button.disabled = True
        leeren_button.disabled = True
        pause_button.disabled = False

    def pause(_):
        global GESTOPPT
        GESTOPPT = True
        play_button.disabled = False
        leeren_button.disabled = False
        pause_button.disabled = True

    def leeren(_):
        global GESTOPPT
        for x in range(GRID_GROSSE):
            for y in range(GRID_GROSSE):
                zelle = SPIELFELD[x, y]
                zustand_festlegen(zelle, "aus")

    play_button.on_click(play)
    pause_button.on_click(pause)
    leeren_button.on_click(leeren)
    display(play_button)
    display(pause_button)
    display(leeren_button)

    display(SPIELFELD)

def tick(zelle, zellen_zustand, nachbarn_zustaende):
    pass

TICK_STEP = 0.5
def main_loop():
    while True:
      sleep(TICK_STEP)
      if GESTOPPT:
          continue
      altem_zustand = ZELLEN_ZUSTAND.copy()
      for x in range(GRID_GROSSE):
        for y in range(GRID_GROSSE):
          zelle = SPIELFELD[x, y]
          nachbarn_zustaende = []
          for nachbar_x in range(x - 1, x + 2):
            for nachbar_y in range(y - 1, y + 2):
              if nachbar_x == x and nachbar_y == y:
                # Das sind uns!!
                continue
              if nachbar_x >= 0 and nachbar_x < GRID_GROSSE and nachbar_y >= 0 and nachbar_y < GRID_GROSSE:
                nachbarn_zustaende.append(altem_zustand[nachbar_x, nachbar_y])
          tick(zelle, altem_zustand[x, y], nachbarn_zustaende)

if not MAIN_LOOP:
    MAIN_LOOP = Thread(target=main_loop)
    MAIN_LOOP.start()

In [2]:
def tick(zelle, altem_zustand, nachbarn_zustaende):
    zustand_festlegen(zelle, "aus")

conways_spiel()

Button(description='Play', layout=Layout(width='80px'), style=ButtonStyle())

Button(description='Pause', disabled=True, layout=Layout(width='80px'), style=ButtonStyle())

Button(description='Leeren', disabled=True, layout=Layout(width='80px'), style=ButtonStyle())

GridspecLayout(children=(Button(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', bo…